In [1]:
import os
import timm
import logging
import argparse
import pandas as pd
import numpy as np
from typing import Optional
from datetime import datetime

import sys
sys.path.append('../')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data
import torch.utils.data.distributed
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset


from utils.dataset import PapsClsDataset, train_transforms, val_transforms, test_transforms, MAX_IMAGE_SIZE
from utils.collate import collate_fn
from utils.sampler import get_weight_random_sampler
# from utils.losses import SupConLoss, FocalLoss

from cls_utils.block import Bottleneck, TwoMLPHead, RoIPool
from collections import Counter

In [2]:
train = pd.read_csv('../lbp_data/train.csv')
train.shape

(17828, 16)

In [3]:
train_dataset = PapsClsDataset(train, transform=train_transforms)

(17828, 16)


In [4]:
train_dataset.df.label_cls.value_counts()

0    8523
1    4066
5    2742
2    1103
3     892
4     502
Name: label_cls, dtype: int64

In [5]:
weightedbatchsampler = get_weight_random_sampler(train_dataset)

In [6]:
weightedbatchsampler

In [9]:
train_dataloader = torch.utils.data.DataLoader(
            dataset=train_dataset,
            batch_size=16,
            shuffle=False,
            collate_fn=collate_fn,
            num_workers=8,
            sampler=weightedbatchsampler,
        )

In [10]:
images, boxes, labels = next(iter(train_dataloader))


In [11]:
count=Counter(labels[:,0].numpy())
count

Counter({0: 4, 5: 2, 1: 7, 4: 2, 2: 1})

In [12]:
print(labels[:,0])

tensor([0, 5, 1, 1, 4, 1, 1, 1, 1, 2, 0, 0, 1, 4, 5, 0])


ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 